# Welcome to lab 4!



In this lab, we will start with a couple of confidence interval problems, and then we will have a large challenge section. In this section, we will give you some problems but not tell you which test or technique to use to solve them. Try to figure it out yourself! 

Start by importing the data set that we used in the last lab. You should be able to find the file on your google drive. If not, here is a link to download it https://www.kaggle.com/aungpyaeap/supermarket-sales. 

In [69]:
import pandas as pd

df = pd.read_csv ('supermarket_sales - Sheet1.csv')

df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### One proportion confidence interval 


#### Create a 95% confidence interval of the the proportion of people who primarily buy health and beauty products. 

In [235]:
import statsmodels.stats.proportion as smp

sample_size = len(df)
people_bought_items = len(df.query("`Product line` == 'Health and beauty'"))

lower, upper = smp.proportion_confint(people_bought_items, sample_size, alpha=0.05, method='normal')
print(f'Lower: {lower} and Upper: {upper}')

Lower: 0.12974806354152094 and Upper: 0.17425193645847906


### Two proportion confidence interval 

#### Create a 90% confidence interval of the difference between the proportion of people who primarily buy health and beauty products and the proportion who buy sports and travel.

In [119]:
'''
The difference between the proportion of people who bought health and beauty vs who bought sports and travel
'''
# Those who bought health and beauty
affected_size1 = len(df.query("`Product line` == 'Health and beauty'"))
sample_size1 = len(df) 

# Those who bought sports and travel
affected_size2 = len(df.query("`Product line` == 'Sports and travel'"))
sample_size2 = len(df)

smp.confint_proportions_2indep(affected_size1,sample_size1,affected_size2,sample_size2,compare='diff', alpha=0.1)


(-0.040917343413215915, 0.01292450294733441)

### One mean confidence interval

Create a 95% confidence interval for the mean Total for the people who buy health and beauty products.

In [120]:
import numpy as np
import scipy.stats as st

degrees_of_freedom = len(df) - 1
average = 323.64301973684223 #df.query("`Product line` == 'Health and beauty'").Total.mean()
std = 237.538815 #df.query("`Product line` == 'Health and beauty'").Total.std()

st.t.interval(alpha=0.1, df=degrees_of_freedom,loc=average, scale=std)


(293.7859835148532, 353.50005595883124)

### Two mean confidence interval

#### Create a 99% confidence interval for the difference between the mean Total for people who buy health and beauty products, and those who buy Sports and travel products. 

In [237]:
import statsmodels.stats.api as sms

df1 = df.query("`Product line` == 'Health and beauty'").Total
df2 = df.query("`Product line` == 'Sports and travel'").Total

cm = sms.CompareMeans(sms.DescrStatsW(df1),sms.DescrStatsW(df2))
#m = sms.CompareMeans(df1, df2)
# Change alpha value
print(cm.tconfint_diff(usevar='unequal'))

(-62.04709679329891, 45.202696507947046)


## Now, let's move on to the challenge problems! 

Start by importing the amrkeing data set. You should be able to find this data set on the class google drive, but if not, you can find a link to download it here: https://www.kaggle.com/jackdaoud/marketing-data. Use df.head() to make sure you can view the data set. 

In [239]:
df2 = pd.read_csv ('marketing_data.csv')

df2.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/14,0,189,...,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,"$57,091.00",0,0,6/15/14,0,464,...,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/14,0,134,...,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/14,0,10,...,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/14,0,6,...,2,7,1,0,0,0,0,1,0,SP


#### We are going to be working a lot with the income variable. Do you notice any problems with the way in which the data is presented that might be problematic for us? How could we solve that problem? 

In [154]:
# The data is in a currency format. We need to convert to float values in order to do calculations
# Also there is a space infront and in the back of Income which is not clear when we printed it out
# Also there are extra spacing in each number
df2.dtypes

ID                      int64
Year_Birth              int64
Education              object
Marital_Status         object
 Income                object
Kidhome                 int64
Teenhome                int64
Dt_Customer            object
Recency                 int64
MntWines                int64
MntFruits               int64
MntMeatProducts         int64
MntFishProducts         int64
MntSweetProducts        int64
MntGoldProds            int64
NumDealsPurchases       int64
NumWebPurchases         int64
NumCatalogPurchases     int64
NumStorePurchases       int64
NumWebVisitsMonth       int64
AcceptedCmp3            int64
AcceptedCmp4            int64
AcceptedCmp5            int64
AcceptedCmp1            int64
AcceptedCmp2            int64
Response                int64
Complain                int64
Country                object
dtype: object

In [155]:
df2[' Income ']

0       $84,835.00 
1       $57,091.00 
2       $67,267.00 
3       $32,474.00 
4       $21,474.00 
           ...     
2235    $66,476.00 
2236    $31,056.00 
2237    $46,310.00 
2238    $65,819.00 
2239    $94,871.00 
Name:  Income , Length: 2240, dtype: object

#### Now, solve it! You can use the function .columns to get the names of all of the columns. 

In [156]:
df2.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', ' Income ',
       'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

In [240]:
df2 = df2.rename(columns={" Income ": "Income"})
df2.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

In [202]:
# Converting all values to floats
df2['Income'].replace('[\$,]', '', regex=True).astype(float)

0       84835.0
1       57091.0
2       67267.0
3       32474.0
4       21474.0
         ...   
2235    66476.0
2236    31056.0
2237    46310.0
2238    65819.0
2239    94871.0
Name: Income, Length: 2240, dtype: float64

#### Use your marketing data to find whether the proportion of people born after 1975 is less than 0.25, at a 5% signifiance level. 

In [184]:
# Should of used hypo test

# Find that less than 25% are born after 1975
num_born_after_75 = len(df2.query("`Year_Birth` > 1975"))
sig_lvl = .05
sample_size = len(df2)

lower, upper = smp.proportion_confint (num_born_after_75,sample_size,alpha= sig_lvl, method='normal')
print(f'Lower: {lower} Upper: {upper}')


Lower: 0.291110627393249 Upper: 0.3294250868924653


#### Use the marketing data to find whether the proportion of people who are divorced is significantly different from the proportion of people who are Married. 

In [191]:
people_divorced = len(df2.query("`Marital_Status` == 'Divorced'"))
people_married = len(df2.query("`Marital_Status` == 'Married'"))
sample_size = len(df2)
print(f'{people_divorced} people who are divorced compared to {people_married} who are married out of {sample_size} total')
# Hypothesis Test? 
'''
H_o = p_1 - p_2 != 0
H_a = p_1 - p_2 == 0
alpha = .10
'''
sample_sucess
from statsmodels.stats.proportion import proportions_ztest
proportions_ztest(count= )

232 people who are divorced compared to 864 who are married out of 2240 total


#### Create a 95% confidence interval to estimate mean Income for your clients. 

In [241]:
import numpy as np
import scipy.stats as st

income = df2['Income'].replace('[\$,]', '', regex=True).astype(float)

mean_income = income.mean()
std_income = income.std()

st.t.interval(alpha=0.05, df=len(df2), loc=mean_income, scale=std_income)

(50668.551948304164, 53825.95075927706)

#### On average, do single people spend more than married people on wine? Use a significance level of .05. 

In [234]:
# Single people one wine

alpha = .05

single_people_wine_avg = df2.query("`Marital_Status` == 'Single'").NumStorePurchases
married_people_wine_avg = df2.query("`Marital_Status` == 'Married'").NumStorePurchases

num_single = df2.query("`Marital_Status` == 'Single'").NumStorePurchases.count()
num_married = df2.query("`Marital_Status` == 'Married'").NumStorePurchases.count()

std_single = df2.query("`Marital_Status` == 'Single'").NumStorePurchases.std()
std_married = df2.query("`Marital_Status` == 'Married'").NumStorePurchases.std()
print(f'Samples: {num_single} {num_married} std: {std_single} {std_single}')


cm = sms.CompareMeans(sms.DescrStatsW(single_people_wine_avg), sms.DescrStatsW(married_people_wine_avg))
print(cm.tconfint_diff(usevar='unequal'))



Samples: 480 864 std: 3.254881645054463 3.254881645054463
(-0.5747142622081507, 0.15249203998592814)


#### Is the mean number of store purchases greater than 6? Check with a significance level of 10%. 

#### Create a 95% confidence interval for the mean income of Divorced people. 